### Robot state and Robot model

**Initialization**

In [25]:
import time
import rclpy
from rclpy.logging import get_logger
#import moveit python library
from moveit.core.robot_state import RobotState
from moveit.planning import (MoveItPy,MultiPipelinePlanRequestParameters)

from moveit.core.planning_scene import PlanningScene

import numpy as np
from geometry_msgs.msg import PoseStamped, Pose    # set pose goal with PoseStamped messsage

from math import pi



def plan_and_execute(
    robot,
    planning_component,
    logger,
    single_plan_parameters=None,
    multi_plan_parameters=None,
    sleep_time=0.0,
):
    """Helper function to plan and execute a motion."""
    # plan to goal
    logger.info("Planning trajectory")
    if multi_plan_parameters is not None:
        plan_result = planning_component.plan(
            multi_plan_parameters=multi_plan_parameters
        )
    elif single_plan_parameters is not None:
        plan_result = planning_component.plan(
            single_plan_parameters=single_plan_parameters
        )
    else:
        plan_result = planning_component.plan()

    # execute the plan
    if plan_result:
        logger.info("Executing plan")
        robot_trajectory = plan_result.trajectory
        robot.execute(robot_trajectory, controllers=[])
    else:
        logger.error("Planning failed")

    time.sleep(sleep_time)


def get_quaternion_from_euler(roll, pitch, yaw):

  qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
  qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
  qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
  qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
 
  return [qx, qy, qz, qw]


# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory

moveit_config = (
        MoveItConfigsBuilder(robot_name="UF_ROBOT", package_name="lite6_enrico")
        .robot_description_semantic(file_path="config/UF_ROBOT.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path="config/UF_ROBOT.urdf.xacro")
        .robot_description_kinematics(file_path="config/kinematics.yaml")
        .joint_limits(file_path="config/joint_limits.yaml")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_moveit_demos")
            + "/config/moveit_cpp.yaml"
        )
        .to_moveit_configs()
    ).to_dict()



# initialise rclpy (only for logging purposes)
#rclpy.init()
logger = get_logger("moveit_py.pose_goal")

# instantiate moveit_py instance and a planning component for the panda_arm
lite6 = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6_arm = lite6.get_planning_component("lite6_arm")

[INFO] [1714493082.411003098] [moveit_3551995780.moveit.py.cpp_initializer]: Initialize node parameters
[INFO] [1714493082.411061238] [moveit_3551995780.moveit.py.cpp_initializer]: Initialize node and executor
[WARN] [1714493082.413239878] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.
[INFO] [1714493082.446359909] [moveit_3551995780.moveit.py.cpp_initializer]: Spin separate thread
[INFO] [1714493082.457176908] [moveit_3551995780.moveit.ros.rdf_loader]: Loaded robot model in 0.0104025 seconds
[INFO] [1714493082.457221822] [moveit_3551995780.moveit.core.robot_model]: Loading robot model 'UF_ROBOT'...
[INFO] [1714493082.457238470] [moveit_3551995780.moveit.core.robot_mod

In [26]:
from moveit.core.robot_model import RobotModel
from moveit.core.robot_state import RobotState

**Below I instantiate a Robotstate object, this is really useful to gather more informations about the robot state**


Representation of a robot’s state. At the lowest level, a state is a collection of variables. Each variable has a name and can have position, velocity, acceleration and effort associated to it. Effort and acceleration share the memory area for efficiency reasons (one should not set both acceleration and effort in the same state and expect things to work). Often variables correspond to joint names as well (joints with one degree of freedom have one variable), but joints with multiple degrees of freedom have more variables. Operations are allowed at variable level, joint level (see JointModel) and joint group level (see JointModelGroup). For efficiency reasons a state computes forward kinematics in a lazy fashion. This can sometimes lead to problems if the update() function was not called on the state.


In [28]:
# Instantiate a RobotState from the robot model
robot_model = lite6.get_robot_model()
robot_state = RobotState(robot_model)

# Set variables
reference_point_position = [0.0,0.0,0.0]
jacobian = []
joint_values = []


In [30]:
model = robot_model.get_joint_model_group("lite6_arm")
# robot_state.get_jacobian("lite6_arm", reference_point_position) it works, i don't know what it means
robot_state.get_jacobian("lite6_arm","link_tcp", reference_point_position, True) # itworks, but results are all the same, even if I change link, why?

array([[0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])

In [ ]:
"""
# I want to instantiate a PlanningScene object kowing that  class moveit.core.planning_scene.PlanningScene(node_name: str, robot_model: moveit.core.robot_model.RobotModel, scene: moveit.core.planning_scene.PlanningSceneInterface, logger: rclpy.logging.Logger = <Logger moveit_py.pose_goal (DEBUG)>)
robot_model = lite6.get_robot_model()
modello = PlanningScene(robot_model=robot_model)
modello.get_frame_transform(frame_id="link_base")
print("modello", modello.get_frame_transform(frame_id="link3"))
"""